In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, max_error
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import skew
import skopt,scipy
import pickle
import sys 

In [2]:
df = pd.read_csv('train_data.csv', on_bad_lines='skip')

In [3]:
rf1 = pickle.load(open('rf1.sav', 'rb'))
rf2 = pickle.load(open('rf2.sav', 'rb'))
im1 = pickle.load(open('im1.sav', 'rb'))

In [4]:
train, test = train_test_split(df, train_size=0.7, random_state=42, shuffle=True)

In [5]:
def delete_column_rows(df, val):
    df = df.drop(['BBLE','OWNER','BLOCK','LOT','BORO','STADDR','POSTCODE','AVLAND2','AVTOT2','EXLAND2','EXTOT2','EXCD2','PERIOD','VALTYPE','Community Board','Census Tract','BIN','NTA','New Georeferenced Column','Council District'], axis = 1)
    val = val.drop(['BBLE','OWNER','BLOCK','LOT','BORO','STADDR','POSTCODE','AVLAND2','AVTOT2','EXLAND2','EXTOT2','EXCD2','PERIOD','VALTYPE','Community Board','Census Tract','BIN','NTA','New Georeferenced Column','Council District'], axis = 1)
    df.loc[((df['LTFRONT'] * df['LTDEPTH']) == 0) & ((df['BLDFRONT'] * df['BLDDEPTH']) == 0),['LTFRONT','LTDEPTH','BLDDEPTH','BLDFRONT']] = np.nan
    val.loc[((val['LTFRONT'] * val['LTDEPTH']) == 0) & ((val['BLDFRONT'] * val['BLDDEPTH']) == 0),['LTFRONT','LTDEPTH','BLDDEPTH','BLDFRONT']] = np.nan
    df['BLDGCL'] = np.where(df['BLDGCL'].isin({'A5', 'R3', 'B2', 'A1', 'C0', 'B9', 'A2', 'B1', 'A9', 'A0', 'C1', 'B3', 'V0', 'R1', 'C7', 'R2'}),
                           df['BLDGCL'], 'Other')
    val['BLDGCL'] = np.where(val['BLDGCL'].isin({'A5', 'R3', 'B2', 'A1', 'C0', 'B9', 'A2', 'B1', 'A9', 'A0', 'C1', 'B3', 'V0', 'R1', 'C7', 'R2'}),
                           val['BLDGCL'], 'Other')
    df['YEAR'] = df['YEAR'].apply(lambda x: x[0:4])
    val['YEAR'] = val['YEAR'].apply(lambda x: x[0:4])
    df['LOTAREA'] = df['LTFRONT'] * df['LTDEPTH']    
    df['BLDAREA'] = df['BLDFRONT'] * df['BLDDEPTH']
    df = df.drop(['LTFRONT','LTDEPTH','BLDFRONT','BLDDEPTH'],axis = 1)
    val['LOTAREA'] = val['LTFRONT'] * val['LTDEPTH']    
    val['BLDAREA'] = val['BLDFRONT'] * val['BLDDEPTH']
    val = val.drop(['LTFRONT','LTDEPTH','BLDFRONT','BLDDEPTH'],axis = 1)
    return df, val

In [6]:
def fit(X_train, X_test, y_train, y_test, rf1, rf2, im1):
    ohe = OneHotEncoder()
    index = X_train.index
    drop = X_train.drop(['EASEMENT','EXT','EXMPTCL','YEAR','Borough','BLDGCL','TAXCLASS'], axis=1)
    X_train = ohe.fit_transform(X_train[['EASEMENT','EXT','EXMPTCL','YEAR','Borough','BLDGCL','TAXCLASS']]).toarray()
    col = ohe.get_feature_names_out(['EASEMENT','EXT','EXMPTCL','YEAR','Borough','BLDGCL','TAXCLASS'])
    X_train = pd.DataFrame(X_train, index = index, columns = col)
    X_train = X_train.join(drop)
    index = X_test.index
    drop = X_test.drop(['EASEMENT','EXT','EXMPTCL','YEAR','Borough','BLDGCL','TAXCLASS'], axis=1)
    X_test = ohe.transform(X_test[['EASEMENT','EXT','EXMPTCL','YEAR','Borough','BLDGCL','TAXCLASS']]).toarray()
    X_test = pd.DataFrame(X_test, index = index, columns = col)
    X_test = X_test.join(drop)
    
    im1 = KNNImputer(n_neighbors=1)
    im1.fit(X_train)
    X_train = pd.DataFrame(im1.transform(X_train.values), columns = X_train.columns, index = X_train.index)
    X_test = pd.DataFrame(im1.transform(X_test.values), columns = X_test.columns, index = X_test.index)
    X_train = X_train.drop(['YEAR_2017','EASEMENT_nan','EXT_nan','EXMPTCL_nan','Borough_nan'], axis = 1)
    X_test = X_test.drop(['YEAR_2017','EASEMENT_nan','EXT_nan','EXMPTCL_nan','Borough_nan'], axis = 1)
    
    cols = ['STORIES','LOTAREA','BLDAREA']
    
    for column in cols:
        high = np.quantile(X_train[column], 0.975)
        X_train.loc[X_train[column] > high, column] = high
        X_test.loc[X_test[column] > high, column] = high
    
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train.values), columns = X_train.columns, index = X_train.index)
    X_test = pd.DataFrame(scaler.transform(X_test.values), columns = X_test.columns, index = X_test.index)
    
    
    rf1.fit(X_train,y_train)
    q = np.quantile(rf1.feature_importances_, 0.5)
    weak_columns = rf1.feature_names_in_[np.where(rf1.feature_importances_ < q)]
    X_train.drop(weak_columns, axis=1, inplace=True)
    X_test.drop(weak_columns, axis=1, inplace=True)
    
    rf2.fit(X_train,y_train)
    importance = rf2.feature_importances_
    col_importance = pd.DataFrame({'col': X_train.columns, 'importance': importance}, columns=['col','importance'])
    corr_matrix = X_train.corr(method='pearson').abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = []
    j=1
    for column in upper.columns:
        if(j == upper.columns.size):
            break
        j+=1
        if any(upper[column]>0.8):
            correlated = upper[upper[column]>0.8].index
            most_important = column
            for col in correlated:
                if col_importance.loc[col_importance.col == col]['importance'].values[0] > col_importance.loc[col_importance.col == most_important]['importance'].values[0]:
                    to_drop.append(most_important)
                    most_important = col
                else:
                    to_drop.append(col)

    corr_matrix = X_train.corr(method='spearman').abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    j=1
    for column in upper.columns:
        if(j == upper.columns.size):
            break
        j+=1
        if any(upper[column]>0.8):
            correlated = upper[upper[column]>0.8].index
            most_important = column
            for col in correlated:
                if col_importance.loc[col_importance.col == col]['importance'].values[0] > col_importance.loc[col_importance.col == most_important]['importance'].values[0]:
                    to_drop.append(most_important)
                    most_important = col
                else:
                    to_drop.append(col)
    X_train.drop(to_drop, axis=1, inplace=True)
    X_test.drop(to_drop, axis=1, inplace=True)
    
    numeric_features = list(X_train.select_dtypes("float64").columns)

    high_skewness = []

    for el in numeric_features:
        if skew(X_train[el])  > 1.5:
            high_skewness.append(el)
    
    X_train[high_skewness] = np.log2(X_train[high_skewness] + sys.float_info.epsilon)
    X_test[high_skewness] = np.log2(X_test[high_skewness] + sys.float_info.epsilon)
#     X_train[high_skewness] = X_train[high_skewness].apply(lambda x: 1/(x + sys.float_info.epsilon))
#     X_test[high_skewness] = X_test[high_skewness].apply(lambda x: 1/(x + sys.float_info.epsilon))
    
    return X_train, X_test, y_train, y_test

In [7]:
def pipe(train, test,rf1,rf2,im1):
    train, test = delete_column_rows(train, test)
    y_train = train['FULLVAL']
    X_train = train.drop('FULLVAL', axis = 1)
    y_test = test['FULLVAL']
    X_test = test.drop('FULLVAL', axis = 1)
    X_train, X_test, y_train, y_test = fit(X_train, X_test, y_train, y_test,rf1,rf2,im1)
    return X_train, X_test, y_train, y_test
    

In [8]:
X_train, X_test, y_train, y_test = pipe(train,test,rf1,rf2,im1)
model = pickle.load(open('finalizedmodel.sav', 'rb'))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(mean_absolute_error(y_pred, y_test))
max_error(y_pred, y_test)

C:\Users\ltoma\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNNImputer was fitted with feature names
  warnings.warn(
C:\Users\ltoma\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNNImputer was fitted with feature names
  warnings.warn(
C:\Users\ltoma\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\ltoma\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
<ipython-input-6-e29a3829bcc5>:45: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated

111977.73344507335


27734937.64